# 多元统计分析作业三（小组）

- 胡义磊 2020010801005
- 张逸斐 2020010801006
- 阮登科 2020010801008
- 蔡与望 2020010801024

导入包，预设参数。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
plt.tight_layout()

## 利率与汇率的关系

读取利率、汇率和 GDP 同比数据集，并清洗数据。

In [ ]:
interest_rate = pd.read_excel(
    "data/加息-汇率-GDP.xlsx",
    sheet_name="加息",
    index_col=0,
)
exchange_rates = pd.read_excel(
    "data/加息-汇率-GDP.xlsx",
    sheet_name="汇率变化",
    index_col=0,
)
gdp_growths = pd.read_excel(
    "data/加息-汇率-GDP.xlsx",
    sheet_name="GDP同比增长率",
    index_col=0,
)
countries = exchange_rates.columns

探索性分析。

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
sns.lineplot(interest_rate, dashes=False, ax=axes[0])
axes[0].set_title("加息")
sns.boxplot(exchange_rates, ax=axes[1])
axes[1].set_title("汇率变化")
sns.boxplot(gdp_growths, ax=axes[2])
axes[2].set_title("GDP同比增长率")
plt.show()

对于各个国家，用加息基点分别和汇率、GDP 同比增长率进行回归分析。

In [ ]:
for index, country in enumerate(countries):
    country_data = pd.DataFrame(
        {
            "加息基点": interest_rate["加息基点"],
            "汇率变化": exchange_rates[country],
            "GDP同比增长率": gdp_growths[country],
        },
        index=interest_rate.index,
    )

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(country)

    sns.heatmap(
        country_data.corr(),
        annot=True,
        cmap="coolwarm",
        vmax=1,
        vmin=-1,
        mask=country_data.corr() == 1,
        ax=axes[0],
    )
    sns.regplot(
        country_data,
        x="加息基点",
        y="汇率变化",
        ax=axes[1],
    )
    sns.regplot(
        country_data,
        x="加息基点",
        y="GDP同比增长率",
        ax=axes[2],
    )
    plt.show()

    print("\n\n回归结果：汇率变化 ~ 加息基点")
    exchange_model = sm.formula.ols(
        formula="汇率变化 ~ 加息基点",
        data=country_data,
    ).fit()
    print(exchange_model.summary())

    print("\n\n回归结果：GDP同比增长率 ~ 加息基点")
    gdp_model = sm.formula.ols(
        formula="GDP同比增长率 ~ 加息基点",
        data=country_data,
    ).fit()
    print(gdp_model.summary())


1. 可以看到，对大多数国家来说，美联储加息对该国的汇率和 GDP 同比增长率没有显著的线性关系。这是由于各国的汇率和 GDP 同比增长率受到的因素很多，例如政策、地缘、技术等等，而美联储加息只是其中的一个因素。
2. 从总体趋势上看，美联储加息使得大多数国家的货币贬值。这是因为美联储加息会使得美元升值，而美元升值会使得其他货币贬值。
3. GDP 同比增长率受美联储加息的影响比较微弱。随着加息基点的增长，有些国家（如韩、英）的 GDP 同比增长率下降，有些国家（如中、非、印）上升，有些国家（如日）保持不变。但 GDP 同比增长率的变化，与其说是受到了美联储加息的影响，不如说是更多受到了国内其它因素的影响。